In [1]:
######## fonctions disponibles : 

# Manipulation : interpolate_space_time, temporel_interpolate, spatial_interpolate, f , f_simple, f_ED, f_ED_invert, get_SReal "profil spatial", get_Profil_Temporel, get_USpoint "profil spatio-temporel"

# Spatial  : min, max, mean, Inside, atPos, intersection, intersect, all_intersections

# Temporel : min, max, mean, atInstant, minInterval, duration

# Spatio-Temporel : Present

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import (StructField,StructType, BooleanType, DoubleType,LongType)
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, to_timestamp, unix_timestamp,udf
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType, FloatType, DataType, TimestampType, Row
import math
import inspect
from pyspark.sql.functions import sqrt
from pyspark.sql.functions import lit

In [3]:
# LECTURE DU FICHIER EN ENTREE

df_data = spark.read.option("header",True).option("inferSchema",True).csv("../VGP-week3-data.csv")

In [4]:
data_schema = df_data.schema
print(data_schema)

StructType(List(StructField(kit_id,IntegerType,true),StructField(participant_id,IntegerType,true),StructField(time,StringType,true),StructField(lat,DoubleType,true),StructField(lon,DoubleType,true),StructField(PM2-5,StringType,true),StructField(PM10,StringType,true),StructField(PM1-0,StringType,true),StructField(NO2,StringType,true),StructField(BC,StringType,true),StructField(activity,StringType,true),StructField(event,StringType,true)))


In [5]:
# RENOMMER LES COLONNES EN CHANGEANT "-" PAR "_"
df_data = df_data.withColumnRenamed('PM2-5','PM2_5')
df_data = df_data.withColumnRenamed('PM1-0','PM1_0')


#On change les string "NULL" en null
df_data = df_data.withColumn(("BC"), when(col("BC") == "NULL", None).otherwise(col("BC")))
df_data = df_data.withColumn(("PM2_5"), when(col("PM2_5") == "NULL", None).otherwise(col("PM2_5")))
df_data = df_data.withColumn(("PM10"),  when(col("PM10") == "NULL",  None).otherwise(col("PM10")))
df_data = df_data.withColumn(("PM1_0"), when(col("PM1_0") == "NULL", None).otherwise(col("PM1_0")))
df_data = df_data.withColumn(("NO2"),   when(col("NO2") == "NULL",   None).otherwise(col("NO2")))
df_data = df_data.withColumn(("activity"), when(col("activity") == "NULL", None).otherwise(col("activity")))
df_data = df_data.withColumn(("event"), when(col("event") == "NULL", None).otherwise(col("event")))


In [6]:
# CONVERSION DE LA DATE EN TIMESTAMP (DE TYPE "LONG")

df_data = df_data.withColumn('time', unix_timestamp('time','yyyy-MM-dd HH:mm:ss').alias('time'))

In [7]:
# VERIFICATION DE LA BONNE CONVERSION DE LA DATE

df_data.where(col('time').isNotNull()).show(5)
df_data.printSchema()

+------+--------------+----------+----------------+--------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|      time|             lat|     lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|
+------+--------------+----------+----------------+--------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|1573718400|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718410|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718420|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718430|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718440|48.7717766666667|2.006005| null|null| null|null|null|    null| null|
+------+--------------+----------+----------------+--------+-----+----+-----+----+----+--------+-----+
only showing top 5 rows

root
 |-- kit_id: integer (nullable = true)
 |--

In [8]:

#######################################################
#######################################################

# DEFINITION DES ADTs

#######################################################
#######################################################


In [9]:
############################################### DEFINITION ADT SPACE-TEMPOREL ###############################################
# TYPE USPOINT
USpoint_schema = [StructField('t1', LongType(),True),
                 StructField('t2', LongType(),True),
                StructField('lat1', DoubleType(),True),
                StructField('long1', DoubleType(),True),
                StructField('lat2', DoubleType(),True),
                StructField('long2', DoubleType(),True),]
USpoint = StructType(fields=USpoint_schema)

# TYPE MREAL (Array(USPOINT))
MSpoint_format = [StructField('time_units', ArrayType(USpoint),True)]
MSpointType = StructType(MSpoint_format)


############################################### DEFINITION ADT TEMPOREL ###############################################
# TYPE INTIME TEMPOREL(val, t1)
intime_schema = [StructField('val', FloatType(),True),
                 StructField('t1', IntegerType(),True)]
intime = StructType(fields=intime_schema)

# TYPE UREAL (a, b, c, r, t1, t2)
Ureal_format = [StructField('a', IntegerType(),True),
              StructField('b', DoubleType(),True),
              StructField('c', DoubleType(),True),
              StructField('r', BooleanType(),True),
              StructField('t1', LongType(),True),
              StructField('t2', LongType(),True)]
UrealType = StructType(Ureal_format)


# TYPE MREAL (Array(UREAL))
Mreal_format = [StructField('time_units', ArrayType(UrealType),True)]
MrealType = StructType(Mreal_format)

############################################### DEFINITION ADT SPATIAL ###############################################

# TYPE POSITION (Latitude, Longitude)
Position_format = [StructField('lat', DoubleType(),True),
              StructField('long', DoubleType(),True)]
PositionType = StructType(Position_format)

# TYPE INTIME SPATIAL(val, latitude, longitude)
intime_schema = [StructField('val', DoubleType(),True),
                 StructField('Lat', DoubleType(),True),
                 StructField('Long', DoubleType(),True)]
intimeSpatial = StructType(fields=intime_schema)

# TYPE USREAL (a, b, c, r, latitude1, longitude1, latitude2, longitude2)
USreal_format = [StructField('a', IntegerType(),True),
              StructField('b', DoubleType(),True),
              StructField('c', DoubleType(),True),
              StructField('r', BooleanType(),True),
              StructField('lat1', DoubleType(),True),
              StructField('long1', DoubleType(),True),
              StructField('lat2', DoubleType(),True),
              StructField('long2', DoubleType(),True)]
USrealType = StructType(USreal_format)


# TYPE MSREAL (Array(USREAL))
MSreal_format = [StructField('space_units', ArrayType(USrealType),True)]
MSrealType = StructType(MSreal_format)

In [10]:
#################################### DEFINITION DE NOS FONCTIONS POUR LE CALCUL DE KEY ####################################

# FONCTION NORME EUCLIDIENNE QUI PREND DES LISTES
def f(longitude_row, latitude_row):
    key_list=[]
    for i in range(len(longitude_row)):
        key = math.sqrt((longitude_row[i]*longitude_row[i]) + (latitude_row[i]*latitude_row[i]))
        key_list.append(key)
    return key_list

# FONCTION NORME EUCLIDIENNE QUI PREND DES TYPES SIMPLES
def f_Simple(longitude_row, latitude_row):
    key = math.sqrt((longitude_row*longitude_row) + (latitude_row*latitude_row))
    return key

# FONCTION ADDITIONNELLE A DETAILLER LORS DE LA PRESENTATION
def f_ED(long, lat, precision_long):
    ent=int(long*pow(10,precision_long))*10
    if long < 0:
        ent=-ent+1
    dec=lat/pow(10,3)
    if dec < 0:
        dec= -dec 
        dec+=pow(10,-1)
    return ent+dec 
f_ED(2.1308233, -48.8014233,3)

21300.1488014233

In [11]:
# INVERSE FONCTION ADDITIONNELLE A DETAILLER LORS DE LA PRESENTATION
def f_ED_invert(key,precision_long):
    ent = int(key)
    if ent % 10 == 1:
        ent = -ent 
    ent = int(ent/10)
    long = ent / pow(10,precision_long)
    dec = key - int(key) 
    if int(dec*10)==1:
        dec = -dec 
    dec = dec*10 - int(dec*10)
    lat = dec * pow(10,2)
    return long,lat 

f_ED_invert(f_ED(-2.1308233, -48.8014233,3),3)
    
    

(-2.13, -48.801423300028546)

In [12]:
#####################################################################################################################
#################################### PARTIE TEMPORELLE ##############################################################
#####################################################################################################################

In [13]:
# DEFINITION DE LA METHODE D'INTERPOLATION POUR LA CREATION D'UN PROFIL TEMPOREL
def interpolate(time_serie_row, measures_row):
    ureal_list=[]
    for i in range((len(measures_row))-1):
            a=0
            if (time_serie_row[i+1]-time_serie_row[i]) == 0:
                i=i+1
            else:
                b=(float(measures_row[i+1])-float(measures_row[i]))/(time_serie_row[i+1]-time_serie_row[i])
                c=((float(measures_row[i])*time_serie_row[i+1])-(float(measures_row[i+1])*time_serie_row[i]))/(time_serie_row[i+1]-time_serie_row[i])
            r=False
            new_ureal = [a,b,c,r,time_serie_row[i],time_serie_row[i+1]]
            ureal_list.append(new_ureal)
    return Row(one_participant_profile = ureal_list)


In [14]:
##############################################################################################################
##############################################################################################################

# MANIPULATION 

# RECUPERATION DU PROFIL TEMPOREL

def get_Profil_Temporel(DF, id_participant, string_mesure_PT):
    DF_Calcul = DF.select('participant_id',string_mesure_PT)
    DF_Calcul = DF_Calcul.filter(DF_Calcul['participant_id'] == id_participant).select(string_mesure_PT)
    DF_Calcul = DF_Calcul.select(DF_Calcul[string_mesure_PT]['time_units'])
    DF_Calcul = DF_Calcul.collect()[0]
    maListe = DF_Calcul[0]
    DF_Result = spark.createDataFrame(maListe,UrealType)
    return DF_Result

##############################################################################################################
##############################################################################################################

# INTERACTION

# INTERVALLE MINIMAL

def minimal_interval(row):
    diff_values=[]
    for i in range(len(row)-1):
        diff_values.append((row[i+1]-row[i]))
    return min(diff_values)


# DURATION GLOBALE DU SUIVI DU PROFILE

def traj_duration(row):
    return (max(row)-min(row))



# RECUPERATION DE LA VALEUR DE LA MESURE A UN TEMPS DONNé DANS UN PROFIL TEMPOREL

def atInstant(MrealDF, t):
        df_operation = MrealDF.filter((col('t1')<= t) & (col('t2')>= t))
        df_operation = df_operation.withColumn('valeur', (df_operation[1]*t) + df_operation[2])
        
        value = df_operation.select(df_operation['valeur']).collect()[0][0]
        
        d = [(value,t)]
        
        result = spark.createDataFrame(d, intime)
        return result

##############################################################################################################
##############################################################################################################

# FONCTIONS 


# VALEUR MOYENNE DU PROFIL TEMPOREL

def Mean_PT(MrealDF):
    
    Mean = MrealDF.rdd.map(lambda x: (((x[0]/3)*math.fabs(math.pow(x[5], 3)-math.pow(x[4], 3)) + (x[1]/2)*math.fabs(math.pow(x[5], 2)-math.pow(x[4], 2)) + x[2]*math.fabs(x[5] - x[4])),(math.fabs(x[5] - x[4]))))
    
    MeanDF = Mean.toDF()
    
    sommeMesures =   MeanDF.select("_1").agg(F.sum(MeanDF._1).alias("somme"))
    sommeInterval =  MeanDF.select("_2").agg(F.sum(MeanDF._2).alias("interval"))
    
    result = sommeMesures.collect()[0][0] / sommeInterval.collect()[0][0]
    
    return result
                             
    
# VALEUR MAXIMALE DU PROFILE TEMPOREL

def Max_PT(MrealDF):
    
    MaxRDD = MrealDF.rdd.map(lambda x: (x[1]*x[5] + x[2]) if (x[1] > 0) else (x[1]*x[4] + x[2]))
    result = MaxRDD.max()
    
    return result


# VALEUR MINIMALE DU PROFILE TEMPOREL

def Min_PT(MrealDF):
    
    MinRDD = MrealDF.rdd.map(lambda x: (x[1]*x[4] + x[2]) if (x[1] > 0) else (x[1]*x[5] + x[2]))
    result = MinRDD.min()
    
    return result



In [15]:
# UDF DE L'INTERPOLATION TEMPORELLE
udfInterpolate = udf(interpolate, MrealType)

# UDF DE L'INTERVALLE MINIMAL
udfMinimalInterval = udf(minimal_interval, IntegerType())

# UDF DE LA DURATION GLOBALE
udfTrajDuration = udf(traj_duration, IntegerType())


In [16]:
# DEBUT MANIPULATION POUR TEST FONCTIONS TEMPORELLES

In [17]:
# NETOYAGE DE L'ENSEMBLE DE DONNEES TEMPORELLE
DataSet = df_data.filter(df_data["time"].isNotNull() & df_data["PM2_5"].isNotNull() & df_data["PM10"].isNotNull() & df_data["PM1_0"].isNotNull() & df_data["NO2"].isNotNull() & df_data["BC"].isNotNull() & df_data["activity"].isNotNull() & df_data["event"].isNotNull() ).select("participant_id","time","PM2_5", "PM10","PM1_0","NO2","BC","activity","event")

# AFFICHAGE DE L'ENSEMBLE APRES NETOYAGE
DataSet.show(5)

+--------------+----------+-----+----+-----+---+---+--------+-------------------+
|participant_id|      time|PM2_5|PM10|PM1_0|NO2| BC|activity|              event|
+--------------+----------+-----+----+-----+---+---+--------+-------------------+
|       9999964|1573804980|    3|   4|    4|  7|376|  Bureau|Arrêter De Cuisiner|
|       9999964|1573804990|    3|   4|    4|  7|376|  Bureau|Arrêter De Cuisiner|
|       9999964|1573805000|    3|   4|    4|  7|376|  Bureau|Arrêter De Cuisiner|
|       9999964|1573805010|    3|   4|    4|  7|376|  Bureau|Arrêter De Cuisiner|
|       9999964|1573805020|    3|   4|    4|  7|376|  Bureau|Arrêter De Cuisiner|
+--------------+----------+-----+----+-----+---+---+--------+-------------------+
only showing top 5 rows



In [18]:
# TEMPORELLE 
# AGGREGATION DES DONNEES GROUPéS PAR PARTICIPANT
DataSet = DataSet.groupBy("participant_id").agg(F.collect_list(col('time')).alias('time'),F.collect_list(col('PM2_5')).alias('PM2_5'),F.collect_list(col('PM10')).alias('PM10'),F.collect_list(col('PM1_0')).alias('PM1_0'),F.collect_list(col('NO2')).alias('NO2'),F.collect_list(col('BC')).alias('BC'),F.collect_list(col('activity')).alias('activity'),F.collect_list(col('event')).alias('event'))

# AFFICHAGE DES DONNES APRES AGGREGATION
DataSet.select("participant_id", "PM2_5", "NO2", "activity", "event").show(5)


+--------------+--------------------+--------------------+--------------------+--------------------+
|participant_id|               PM2_5|                 NO2|            activity|               event|
+--------------+--------------------+--------------------+--------------------+--------------------+
|       9999960|[9, 7, 7, 9, 9, 8...|[15, 13, 13, 12, ...|[Bureau, Bureau, ...|[Fermeture De Fen...|
|       9999964|[3, 3, 3, 3, 3, 3...|[7, 7, 7, 7, 7, 7...|[Bureau, Bureau, ...|[Arrêter De Cuisi...|
+--------------+--------------------+--------------------+--------------------+--------------------+



In [19]:
# CREATION DU PROFIL TEMPOREL AVEC L'INTERPOLATION
DataSet_temporal_Profiles = DataSet
DataSet_temporal_Profiles = DataSet_temporal_Profiles.withColumn('PM2_5_PT', udfInterpolate('time','PM2_5'))
DataSet_temporal_Profiles = DataSet_temporal_Profiles.withColumn('PM10_PT', udfInterpolate('time','PM10'))
DataSet_temporal_Profiles = DataSet_temporal_Profiles.withColumn('PM1_0_PT', udfInterpolate('time','PM1_0'))
DataSet_temporal_Profiles = DataSet_temporal_Profiles.withColumn('NO2_PT', udfInterpolate('time','NO2'))
DataSet_temporal_Profiles = DataSet_temporal_Profiles.withColumn('BC_PT', udfInterpolate('time','BC'))

DataSet_temporal_Profiles = DataSet_temporal_Profiles.select("participant_id", "time", "PM2_5_PT", "PM10_PT", "PM1_0_PT", "NO2_PT", "BC_PT")

DataSet_temporal_Profiles.show()

+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|participant_id|                time|            PM2_5_PT|             PM10_PT|            PM1_0_PT|              NO2_PT|               BC_PT|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       9999960|[1574064040, 1574...|[[[0, -0.06666666...|[[[0, -0.06666666...|[[[0, -0.06666666...|[[[0, -0.06666666...|[[[0, 1.533333333...|
|       9999964|[1573804980, 1573...|[[[0, 0.0, 3.0, f...|[[[0, 0.0, 4.0, f...|[[[0, 0.0, 4.0, f...|[[[0, 0.0, 7.0, f...|[[[0, 0.0, 376.0,...|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [20]:
########################################### TEST FONCTIONS TEMPORELLES  ###########################################
# RECUPEARION DU PROFIL TEMPOREL D'UN PARTICIPANT

id_participant = 9999964
mesure = "PM2_5_PT"
PM2_5_PT_DF = get_Profil_Temporel(DataSet_temporal_Profiles, id_participant, mesure)

# RECUPERATION DE LA VALEUR MOYENNE DU PROFIL TEMPOREL DE LA MESURE "PM2_5"
mean_PM2_5 = Mean_PT(PM2_5_PT_DF)
print(mean_PM2_5)


# RECUPERATION DE LA VALEUR MAXIMALE DU PROFIL TEMPOREL DE LA MESURE "PM2_5"
max_PM2_5  = Max_PT(PM2_5_PT_DF)
print(max_PM2_5)


# RECUPERATION DE LA VALEUR MINIMALE DU PROFIL TEMPOREL DE LA MESURE "PM2_5"
min_PM2_5  = Min_PT(PM2_5_PT_DF)
print(min_PM2_5)



5.3892250866017495
23.0
2.0


In [21]:
########################################### TEST FONCTIONS TEMPORELLES SUITE ###########################################

# INTERVALLE MINIMAL
DataSet_Profiles_Minimal = DataSet_temporal_Profiles.withColumn('minimal', udfMinimalInterval('time'))
DataSet_Profiles_Minimal.select('participant_id','minimal').show()

# DURATION
#DataSet_Profiles.show()
DataSet_Profiles_Duration = DataSet_temporal_Profiles.withColumn('duration', udfTrajDuration('time'))
DataSet_Profiles_Duration.select('participant_id','duration').show()

# TEST FONTION atINSTANT() ---> RECUPERATION DE LA VALEUR DE "PM2_5" A L'INSTANT 't = 1573810620'
t = 1573810620
result = atInstant(PM2_5_PT_DF, t)
result.show()



+--------------+-------+
|participant_id|minimal|
+--------------+-------+
|       9999960|     30|
|       9999964|     10|
+--------------+-------+

+--------------+--------+
|participant_id|duration|
+--------------+--------+
|       9999960|  194720|
|       9999964|  528670|
+--------------+--------+

+---+----------+
|val|        t1|
+---+----------+
|3.0|1573810620|
+---+----------+



In [22]:
#####################################################################################################################
#################################### PARTIE SPATIAL ##############################################################
#####################################################################################################################

In [23]:
# DEFINITION DE LA METHODE D'INTERPOLATION POUR LA CREATION D'UN PROFIL SPATIAL
def spatial_interpolate(Key_serie_row, measures_row, longitude_row, latitude_row):
    usreal_list=[]
    for i in range(len(Key_serie_row)-2):
            a=0
            while (i<len(Key_serie_row)-2) and (Key_serie_row[i+1]-Key_serie_row[i]) == 0 :
                i=i+1
            
            b=(float(measures_row[i+1])-float(measures_row[i]))/(Key_serie_row[i+1]-Key_serie_row[i])
            c=((float(measures_row[i])*Key_serie_row[i+1])-(float(measures_row[i+1])*Key_serie_row[i]))/(Key_serie_row[i+1]-Key_serie_row[i])
            x1=[(latitude_row[i], longitude_row[i])]
            x2=[(latitude_row[i+1], longitude_row[i+1])]
            lat1 = latitude_row[i]
            long1 = longitude_row[i]
            lat2 = latitude_row[i+1]
            long2 = longitude_row[i+1]
            r=False
            new_usreal = [a,b,c,r,lat1,long1,lat2,long2]
            usreal_list.append(new_usreal)
    return Row(one_participant_profile = usreal_list)

In [24]:
##############################################################################################################
##############################################################################################################

# MANIPULATION 

# RECUPERATION DU PROFIL SPATIAL

# FONCTION QUI RETOURNE LE PROFIL SPATIAL D UN PARTICIPANT --> A changer plus tard avec la Version2
def get_SReal(DF, id_participant):
    toto = DF.select('participant_id','profil_spatial')
    a = toto.filter(toto['participant_id'] == id_participant).select('profil_spatial')
    b = a.select(a['profil_spatial']['space_units'])
    c = b.collect()[0]
    maListe = c[0]
    tata = spark.createDataFrame(maListe,USrealType)
    return tata

##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################


# FONCTIONS 

# FONCTION MEAN() PROFIL SPATIAL
def mean_PS(ProfilSpatial):
    interval = 0
    summ = 0
    for row in profilSpatial.collect():
        a = row["a"]
        b = row["b"]
        c = row["c"]
        r = row["r"]
        key1 = f_Simple(row["lat1"], row["long1"])
        key2 = f_Simple(row["lat2"], row["long2"])
        i = key1-key2
        if i < 0:
            i = i*(-1)
        interval = interval + i
        summ = summ + (a/3)*math.fabs(math.pow(key2, 3)-math.pow(key1, 3)) + (b/2)*math.fabs(math.pow(key2, 2)-math.pow(key1, 2)) + c*math.fabs(key2 - key1)
    return(summ/interval)
                                   

# FONCTION MAX() PROFIL SPATIAL
def max_PS(profilSpatial):
    maximum = -9999999
    for row in profilSpatial.collect():
        a = row["a"]
        b = row["b"]
        c = row["c"]
        r = row["r"]
        key1 = f_Simple(row["lat1"], row["long1"])
        key2 = f_Simple(row["lat2"], row["long2"])
        if (key1 > key2):
            temp = key1
            key1 = key2
            key2 = temp
        if(a == 0):
            if (b>0):
                temp = a*key2*key2 + b*key2 + c
                if (maximum < temp):
                    maximum = temp
            else:
                temp = a*key1*key1 + b*key1 + c
                if (maximum < temp):
                    maximum = temp
    return maximum


# FONCTION MIN() PROFIL SPATIAL
def min_PS(profilSpatial):
    minimum = 999999999
    for row in profilSpatial.collect():
        a = row["a"]
        b = row["b"]
        c = row["c"]
        r = row["r"]
        key1 = f_Simple(row["lat1"], row["long1"])
        key2 = f_Simple(row["lat2"], row["long2"])
        if (key1 > key2):
            temp = key1
            key1 = key2
            key2 = temp
        if(a == 0):
            if (b<0):
                temp = a*key2*key2 + b*key2 + c
                if (minimum > temp):
                    minimum = temp
            else:
                temp = a*key1*key1 + b*key1 + c
                if (minimum > temp):
                    minimum = temp
    return minimum

# FONCTION INSIDE() PROFIL SPATIAL
# A ameliorer V2 --> faut retourner le segment exact
def inside(profilSpatial, unPoint):
    profilSpatial = profilSpatial.withColumn(("maxLat"), when(col("lat1") <= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("maxLong"), when(col("long1") <= col('long2'), col('long2')).otherwise(col("long1")))
    profilSpatial = profilSpatial.withColumn(("minLat"), when(col("lat1") >= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("minLong"), when(col("long1") >= col('long2'), col('long2')).otherwise(col("long1")))
    result = profilSpatial.filter(((profilSpatial['lat2'] - profilSpatial['lat1']) / (profilSpatial['long2'] - profilSpatial['long1'])) == ((unPoint.head()['lat']-profilSpatial['lat1']) / (unPoint.head()['long']-profilSpatial['long1']))).filter(
    unPoint.head()['lat'] <= profilSpatial['maxLat']).filter(unPoint.head()['lat'] >= profilSpatial['minLat']).filter(
    unPoint.head()['long'] <= profilSpatial['maxLong']).filter(unPoint.head()['long'] >= profilSpatial['minLong'])
    if(result.rdd.isEmpty()):
        return 0
    else: 
        return 1

    
    
# FONCTION atPOS() PROFIL SPATIAL
def atPos(profilSpatial, unPoint):
    
    profilSpatial = profilSpatial.withColumn(("maxLat"), when(col("lat1") <= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("maxLong"), when(col("long1") <= col('long2'), col('long2')).otherwise(col("long1")))
    profilSpatial = profilSpatial.withColumn(("minLat"), when(col("lat1") >= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("minLong"), when(col("long1") >= col('long2'), col('long2')).otherwise(col("long1")))
    result = profilSpatial.filter(((profilSpatial['lat2'] - profilSpatial['lat1']) / (profilSpatial['long2'] - profilSpatial['long1'])) == ((unPoint.head()['lat']-profilSpatial['lat1']) / (unPoint.head()['long']-profilSpatial['long1']))).filter(
    unPoint.head()['lat'] <= profilSpatial['maxLat']).filter(unPoint.head()['lat'] >= profilSpatial['minLat']).filter(
    unPoint.head()['long'] <= profilSpatial['maxLong']).filter(unPoint.head()['long'] >= profilSpatial['minLong'])
    if(result.rdd.isEmpty()):
        return "Le point n'est pas sur notre trajectoire"
    else: 
        #lattList_NotrePoint = [unPoint[0]]
        #longList_NotrePoint = [unPoint[1]]
        val = (result.head()['a'] * f_Simple(unPoint.head()['long'],unPoint.head()['lat'])* f_Simple(unPoint.head()['long'],unPoint.head()['lat'])) + (result.head()['b'] * f_Simple(unPoint.head()['long'],unPoint.head()['lat'])) + result.head()['c'] 
        r = [(val, unPoint.head()['lat'], unPoint.head()['long'])]
        resultFinal = spark.createDataFrame(r, intimeSpatial)
        return r

    
# FONCTION INTERSECTION() --> ENTRE 2 SEGEMENTS
def intersection(USreal1,USreal2):
    segment1= [USreal1.__getattr__('lat1'),USreal1.__getattr__('long1'),USreal1.__getattr__('lat2'),USreal1.__getattr__('long2')]
    segment2= [USreal2.__getattr__('lat1'),USreal2.__getattr__('long1'),USreal2.__getattr__('lat2'),USreal2.__getattr__('long2')]
    if (segment1[0]!=segment1[2]):
        a1,b1=(segment1[1]-segment1[3])/(segment1[0]-segment1[2]),segment1[1]-((segment1[1]-segment1[3])/(segment1[0]-segment1[2]))*segment1[0]
    else : 
        a1,b1 = 999999999999,segment2[1]-999999999999*segment2[0]    
    if (segment2[0]!=segment2[2]):
        a2,b2=(segment2[1]-segment2[3])/(segment2[0]-segment2[2]),segment2[1]-((segment2[1]-segment2[3])/(segment2[0]-segment2[2]))*segment2[0]
    else :
        a2,b2 = 999999999999,segment2[1]-999999999999*segment2[0]   
    if (a1 != a2):
        x0 = (b2-b1)/(a1-a2)
        y0 = (a1*b2-a2*b1)/(a1-a2)
        if (x0>=max(min(segment1[0],segment1[2]),min(segment2[0],segment2[2]))) and (x0<=min(max(segment1[0],segment1[2]),max(segment1[0],segment1[2]))):
            return (x0,y0)
        else:
            return "intersection_hors_segment"
    elif b1==b2:
        return "segment_identique", [max(min(segment1[0],segment1[2]),min(segment2[0],segment2[2])),min(max(segment1[0],segment1[2]),max(segment1[0],segment1[2]))]
    else:
        return "intersection_impossible" 
# FONCTION INTERSECT() --> ENTRE 2 TRAJECTOIRES --> return BOOLEAN            
def intersect(MSreal_row1, MSreal_row2):
    list_MSreal1=MSreal_row1.__getattr__('profil_spatial').__getattr__("space_units")
    list_MSreal2=MSreal_row2.__getattr__('profil_spatial').__getattr__("space_units")
    for seg1 in list_MSreal1:
        for seg2 in list_MSreal2:
            if intersection(seg1,seg2) != "intersection_hors_segment" and  intersection(seg1,seg2) != "intersection_impossible" :
                return 1
            
# FONCTION ALL_INTERSECT() --> ENTRE 2 TRAJECTOIRES --> return ALL POINTS            
def all_intersections(MSreal_row1, MSreal_row2):
    list_MSreal1=MSreal_row1.__getattr__('profil_spatial').__getattr__("space_units")
    list_MSreal2=MSreal_row2.__getattr__('profil_spatial').__getattr__("space_units")
    list_intersect=[]
    for seg1 in list_MSreal1:
        for seg2 in list_MSreal2:
            if intersection(seg1,seg2) != "intersection_hors_segment" and  intersection(seg1,seg2) != "intersection_impossible" :
                list_intersect.append(intersection(seg1,seg2))
    return list_intersect

In [25]:
# UDF KEYS
udfKey = udf(f, ArrayType(DoubleType()))

# UDF DE L'INTERPOLATION SPATIAL
udfProfilSpatial = udf(spatial_interpolate, MSrealType)

In [26]:
# DEBUT MANIPULATION POUR TEST FONCTIONS SPATIALES 

In [27]:
# NETOYAGE DE L'ENSEMBLE DE DONNEES SPATIAL
Dataset_Space_non_null = df_data.filter(df_data["PM1_0"].isNotNull()).filter(df_data["lon"].isNotNull()).filter(df_data["lat"].isNotNull()).select("participant_id","lat","lon","PM1_0")
 
# AGGREGATION DES DONNEES GROUPéS PAR PARTICIPANT
Dataset_Space_non_null = Dataset_Space_non_null.groupBy("participant_id").agg(F.collect_list(col('lat')).alias('lat'), F.collect_list(col('lon')).alias('lon'),F.collect_list(col("PM1_0")).alias("PM1_0"))

# AJOUT DE LA COLONNE KEY
udfKey = udf(f, ArrayType(DoubleType()))
DataSet_aggregation_keys = Dataset_Space_non_null.withColumn('keys', udfKey('lat','lon'))
DataSet_aggregation_keys.show()

# CREATION PROFIL SPATIAL AVEC L'INTERPOLATION POUR LA COLONNE PM1_0
udfProfilSpatial = udf(spatial_interpolate, MSrealType)
DataSet_Spatial_Profiles = DataSet_aggregation_keys.withColumn('profil_spatial', udfProfilSpatial("keys","PM1_0","lon","lat"))

DataSet_Spatial_Profiles.show()

+--------------+--------------------+--------------------+--------------------+--------------------+
|participant_id|                 lat|                 lon|               PM1_0|                keys|
+--------------+--------------------+--------------------+--------------------+--------------------+
|       9999920|[48.8014233333333...|[2.13082333333333...|[19, 19, 16, 16, ...|[48.8479204003311...|
|       9999955|[48.8012116666667...|[2.13086166666667...|[3, 3, 3, 3, 3, 3...|[48.8477106073281...|
|       9999975|[48.8067434, 48.8...|[2.1306277, 2.132...|[25, 25, 21, 21, ...|[48.8532268710005...|
|       9999960|[48.8582383333333...|[2.34773166666667...|[10, 8, 8, 9, 9, ...|[48.9146123056849...|
|       9999964|[48.7717266666667...|[2.00586333333333...|[4, 4, 4, 4, 4, 4...|[48.8129573961674...|
|       9999962|[48.799435, 48.79...|[2.14138166666667...|[7, 7, 7, 7, 7, 7...|[48.8463956885414...|
+--------------+--------------------+--------------------+--------------------+------------

In [28]:
########################################### TEST FONCTIONS TEMPORELLES  ###########################################
# RECUPEARION DU PROFIL TEMPOREL D'UN PARTICIPANT

id_participant = 9999964
profilSpatial = get_SReal(DataSet_Spatial_Profiles,id_participant)

# RECUPERATION DE LA VALEUR MOYENNE DU PROFIL TEMPOREL DE LA MESURE "PM2_5"
#TEST DU MEAN() SPATIAL
#profilSpatial = get_SReal(9999920)
print(mean_PS(profilSpatial))


# RECUPERATION DE LA VALEUR MAXIMALE DU PROFIL TEMPOREL DE LA MESURE "PM2_5"
print(max_PS(profilSpatial))


# RECUPERATION DE LA VALEUR MINIMALE DU PROFIL TEMPOREL DE LA MESURE "PM2_5"
print(min_PS(profilSpatial))



6.9352809505230315
26.000000022351742
0.9999999990686774


In [29]:
########################################### TEST FONCTIONS TEMPORELLES SUITE ###########################################

id_participant = 9999920
profilSpatial = get_SReal(DataSet_Spatial_Profiles,id_participant)

# TEST INSIDE()
pointTest = [48.801415, 2.130805]
sPOINT = spark.createDataFrame(sc.parallelize([pointTest]),PositionType)
print(inside(profilSpatial, sPOINT))

pointTest = [48.801415, 2.130805]
sPOINT = spark.createDataFrame(sc.parallelize([pointTest]),PositionType)
atPos(profilSpatial, sPOINT)


1


[(18.999999999221334, 48.801415, 2.130805)]

In [30]:
########################################### TEST FONCTIONS TEMPORELLES SUITE ########################################### 
MSR1= DataSet_Spatial_Profiles.filter(DataSet_Spatial_Profiles['participant_id']== 9999920).select('profil_spatial').rdd.collect()[0]
MSR2= DataSet_Spatial_Profiles.filter(DataSet_Spatial_Profiles['participant_id']== 9999955).select('profil_spatial').rdd.collect()[0]
#tester la fonction intersection pour deux segment USReal1 et USReal2 d'un seule MSReal1
USR1=MSR1.__getattr__('profil_spatial').__getattr__("space_units")[0]
USR2=MSR1.__getattr__('profil_spatial').__getattr__("space_units")[1]
#tester sur deux segments dont l'intersection est (1.5,1.5) (facile à manipuler pour les tests)
x1 = Row(lat1=1,long1=1,lat2=2,long2=2)
x2 = Row(lat1=1,long1=2,lat2=2,long2=1)
intersection(USR1,USR2),intersection(x1,x2)

((48.801415, 2.130804999999998), (1.5, 1.5))

In [31]:
#segment identique 
intersection(USR1,USR1)

('segment_identique', [48.801415, 48.8014233333333])

In [32]:
#tester le cas d'une intersection hors segment
x1 = Row(lat1=1,long1=1,lat2=2,long2=2)
x2 = Row(lat1=1,long1=3,lat2=2,long2=3)
intersection(x1,x2)

'intersection_hors_segment'

In [33]:
#tester si MSreal1 et MSreal2 ont des trajectoires qui s'intersectent ou pas 
intersect(MSR1,MSR2)

1

In [34]:
# trouver toutes les intersection entre MSreal1 et MSreal2 
# ! à ne pas réexecuter : sa prends énormément de temps à cause de la taille des données et l'absence d'un index
all_intersections(MSR1,MSR2)

[(48.801415000000134, 2.1308050000002914),
 (48.80142164185834, 2.1308196120884038),
 (48.80142079710127, 2.1308177536228667),
 (48.801417380952664, 2.130810238095872),
 (48.801415677506746, 2.1308064905148343),
 (48.80142146067411, 2.130819213483111),
 (48.80142146067411, 2.130819213483111),
 (48.801419817275736, 2.130815598006652),
 (48.801419817275736, 2.130815598006652),
 (48.801418990610316, 2.1308137793427315),
 (48.80141927295919, 2.130814400510249),
 (48.801420313351485, 2.1308166893733254),
 (48.801416436781615, 2.130808160919548),
 (48.801421202588116, 2.1308186456939073),
 (48.801421202588116, 2.1308186456939073),
 (48.801421202588116, 2.1308186456939073),
 (48.80142183760699, 2.130820042735433),
 (48.8014185367545, 2.130812780859922),
 (48.8014185367545, 2.130812780859922),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (4

In [35]:
#####################################################################################################################
#################################### PARTIE SPATIO-TEMPOREL #########################################################
#####################################################################################################################

In [36]:
# DEFINITION DE LA METHODE D'INTERPOLATION SPACE + TEMPS 
def interpolate_space_time(time_serie_row,latitude_row, longitude_row):
    uspoint_list=[]
    for i in range((len(time_serie_row))-1):
            a=0
            if (time_serie_row[i+1]-time_serie_row[i]) == 0:
                i=i+1
            new_uspoint = [time_serie_row[i],time_serie_row[i+1], latitude_row[i], longitude_row[i], latitude_row[i+1], longitude_row[i+1]]
            uspoint_list.append(new_uspoint)
    return Row(one_participant_profile = uspoint_list)


In [37]:
##############################################################################################################
##############################################################################################################

# MANIPULATION 

# RECUPERATION DU PROFIL SPATIO-TEMPOREL --> A changer plus tard avec la Version2


def get_USpoint(DF, id_participant):
    toto = DF.select('participant_id','space_time')
    a = toto.filter(toto['participant_id'] == id_participant).select('space_time')
    b = a.select(a['space_time']['time_units'])
    c = b.collect()[0]
    maListe = c[0]
    tata = spark.createDataFrame(maListe,USpoint)
    return tata

##############################################################################################################
##############################################################################################################

# INTERACTION

# INTERVALLE MINIMAL

def minimal_interval(row):
    diff_values=[]
    for i in range(len(row)-1):
        diff_values.append((row[i+1]-row[i]))
    return min(diff_values)


# DURATION GLOBALE DU SUIVI DU PROFILE

def traj_duration(row):
    return (max(row)-min(row))



# RECUPERATION DE LA VALEUR DE LA MESURE A UN TEMPS DONNé DANS UN PROFIL TEMPOREL

def atInstant(MrealDF, t):
        df_operation = MrealDF.filter((col('t1')<= t) & (col('t2')>= t))
        df_operation = df_operation.withColumn('valeur', (df_operation[1]*t) + df_operation[2])
        
        value = df_operation.select(df_operation['valeur']).collect()[0][0]
        
        d = [(value,t)]
        
        result = spark.createDataFrame(d, intime)
        return result

##############################################################################################################
##############################################################################################################

# FONCTIONS 


# FONCTION PRESENT() PROFIL SPATIO-TEMPOREL
def present(ProfilSpatio_Temporel, time):
    for row in ProfilSpatio_Temporel.collect():
        t1 = row["t1"]
        t2 = row["t2"]
        if ((t1<= time) and (t2>=time)):
            return 1
        else: 
            return 0


In [38]:
# UDF DE L'INTERPOLATION SPACE + TIME
udfInterpolate_space_time = udf(interpolate_space_time, MSpointType)

In [39]:
# DEBUT MANIPULATION POUR TEST FONCTIONS SPATIO-TEMPOREL

In [40]:
# NETOYAGE DE L'ENSEMBLE DE DONNEES SPACE + TIME 
DataSet_space_time = df_data.filter(df_data["time"].isNotNull()).filter(df_data["lon"].isNotNull()).filter(df_data["lat"].isNotNull()).select("participant_id","time","lat","lon")

# AGGREGATION DES DONNEES GROUPéS PAR PARTICIPANT
DataSet_space_time = DataSet_space_time.groupBy("participant_id").agg(F.collect_list(col("time")).alias("time"),F.collect_list(col('lat')).alias('lat'), F.collect_list(col('lon')).alias('lon'))

# CREATION DU PROFIL SPACE + TIME AVEC L'INTERPOLATION
DataSet_space_time = DataSet_space_time.withColumn('space_time', udfInterpolate_space_time('time','lat', 'lon'))
DataSet_space_time = DataSet_space_time.select("participant_id", 'space_time')

DataSet_space_time.show()

+--------------+--------------------+
|participant_id|          space_time|
+--------------+--------------------+
|       9999920|[[[1573810600, 15...|
|       9999955|[[[1573804800, 15...|
|       9999975|[[[1573807550, 15...|
|       9999936|[[[1573804800, 15...|
|       9999930|[[[1573804800, 15...|
|       9999960|[[[1574064040, 15...|
|       9999964|[[[1573718400, 15...|
|       9999962|[[[1573804800, 15...|
|        999992|[[[1573807860, 15...|
+--------------+--------------------+



In [41]:
########################################### TEST FONCTIONS SPATIO-TEMPOREL  ###########################################
# RECUPEARION DU PROFIL SPATIO-TEMPOREL D'UN PARTICIPANT

id_participant = 9999920
profilSpatio_temporel = get_USpoint(DataSet_space_time, id_participant)

#TEST DU PRESENT() SPATIO-TEMPOREL
present(profilSpatio_temporel, 1573810610)

1